---
title: "EDA"
format: html
---

In [1]:
import os
print("Current working directory:", os.getcwd())
print("Files in data/:", os.listdir("./data"))

Current working directory: /Users/prathamkabra/ad688-employability-sp25A1-group11-3
Files in data/: ['eda_data.csv', 'lightcast_job_postings.csv']


In [ ]:
import pandas as pd
eda = pd.read_csv('./data/eda_data.csv')
#eda = pd.read_csv("./data/lightcast_job_postings.csv")
#eda.head(10)

In [ ]:
# identifying data analyst jobs by keyword searching
keywords = ['Data Analyst', 'Business Analyst', 'Data Engineering', 'Deep Learning',
            'Data Science', 'Data Analysis','Data Analytics',  'Market Research Analyst' 
            'LLM', 'Language Model', 'NLP', 'Natural Language Processing',
            'Computer Vision', 'Business Intelligence Analyst', 'Quantitative Analyst', 'Operations Analyst']

match = lambda col: eda[col].str.contains('|'.join(keywords), case=False, na=False)

eda['DATA_ANALYST_JOB'] = match('TITLE_NAME') \
             | match('SKILLS_NAME') \
             | match('SPECIALIZED_SKILLS_NAME') 
eda['DATA_ANALYST_JOB'].value_counts()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

# Group data
df_grouped = eda.groupby(['DATA_ANALYST_JOB', 'NAICS2_NAME']).size().reset_index(name='Job_Count')

# Shorten the industry names
industry_short_names = {
    'Professional, Scientific, and Technical Services': 'Prof. Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Admin & Waste Mgmt',
    'Health Care and Social Assistance': 'Healthcare',
    'Finance and Insurance': 'Finance',
    'Information': 'Info Tech',
    'Educational Services': 'Education',
    'Manufacturing': 'Manufacturing',
    'Retail Trade': 'Retail',
    'Accommodation and Food Services': 'Hospitality',
    'Other Services (except Public Administration)': 'Other Services',
}
df_grouped['NAICS2_NAME_SHORT'] = df_grouped['NAICS2_NAME'].map(industry_short_names).fillna(df_grouped['NAICS2_NAME'])

# Map DATA_ANALYST_JOB to True/False for the legend
df_grouped['Job_Type'] = df_grouped['DATA_ANALYST_JOB'].map({True: 'True', False: 'False'})

# Define gradient color palettes for True/False
color_gradients = {
    'True': ['#4ECDC4', '#6BE6DE'],  # Teal gradient
    'False': ['#FF6B6B', '#FF8787']  # Coral red gradient
}

# Create a subplot with a chart and a table
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.8, 0.2],
    specs=[[{"type": "bar"}], [{"type": "table"}]],
    vertical_spacing=0.05
)

# Create the initial bar chart traces
for job_type in df_grouped['Job_Type'].unique():
    df_subset = df_grouped[df_grouped['Job_Type'] == job_type]
    fig.add_trace(
        go.Bar(
            x=df_subset['NAICS2_NAME_SHORT'],
            y=df_subset['Job_Count'],
            name=job_type,
            marker=dict(
                colorscale=[[0, color_gradients[job_type][0]], [1, color_gradients[job_type][1]]],
                line=dict(width=1.2, color="#2D3748")
            ),
            opacity=0.85,
            text=df_subset['Job_Count'],
            textposition='outside',
            textfont=dict(size=12, color="#2D3748", family="Helvetica, sans-serif"),
        ),
        row=1, col=1
    )

# Add a table below the chart
fig.add_trace(
    go.Table(
        header=dict(
            values=['Industry', 'Job Type', 'Number of Jobs'],
            font=dict(size=14, color="#2D3748", family="Helvetica, sans-serif"),
            fill_color='rgba(240, 240, 245, 1)',
            line_color="#2D3748"
        ),
        cells=dict(
            values=[
                df_grouped['NAICS2_NAME_SHORT'],
                df_grouped['Job_Type'],
                df_grouped['Job_Count']
            ],
            font=dict(size=12, color="#2D3748", family="Helvetica, sans-serif"),
            fill_color='rgba(255, 255, 255, 1)',
            line_color="#2D3748"
        )
    ),
    row=2, col=1
)

# Define themes
light_theme = {
    'plot_bgcolor': 'rgba(240, 240, 245, 1)',
    'paper_bgcolor': 'rgba(255, 255, 255, 1)',
    'font_color': '#2D3748',
    'grid_color': 'rgba(200, 200, 200, 0.2)',
    'line_color': '#2D3748',
    'bg_gradient': 'linear-gradient(135deg, #E6E9FF 0%, #F5F5FA 100%)'
}

dark_theme = {
    'plot_bgcolor': 'rgba(30, 30, 40, 1)',
    'paper_bgcolor': 'rgba(20, 20, 30, 1)',
    'font_color': '#E2E8F0',
    'grid_color': 'rgba(255, 255, 255, 0.1)',
    'line_color': '#E2E8F0',
    'bg_gradient': 'linear-gradient(135deg, #2D3748 0%, #1A202C 100%)'
}

# Initial theme
current_theme = light_theme

# Beautify the layout with advanced interactivity
fig.update_layout(
    # Sizing and margins
    autosize=True,
    width=1200,
    height=900,
    margin=dict(l=50, r=50, t=120, b=50),

    # Background with animated gradient
    shapes=[
        dict(
            type="rect",
            x0=0, y0=0, x1=1, y1=1,
            xref="paper", yref="paper",
            fillcolor=current_theme['bg_gradient'],
            line=dict(width=0),
            layer="below"
        )
    ],
    plot_bgcolor=current_theme['plot_bgcolor'],
    paper_bgcolor=current_theme['paper_bgcolor'],
    font=dict(family="Helvetica, sans-serif", size=14, color=current_theme['font_color']),

    # Title styling
    title=dict(
        text="Data & Business Analytics Job Trends",
        font=dict(size=30, color=current_theme['font_color'], family="Helvetica, sans-serif"),
        x=0.5,
        xanchor="center",
        y=0.95,
        yanchor="top"
    ),

    # X-axis styling
    xaxis=dict(
        title="Industry",
        title_font=dict(size=18, color=current_theme['font_color']),
        tickfont=dict(size=13, color=current_theme['font_color']),
        tickangle=-30,
        gridcolor=current_theme['grid_color'],
        linecolor=current_theme['line_color'],
        linewidth=2,
        showline=True
    ),

    # Y-axis styling
    yaxis=dict(
        title="Number of Jobs",
        title_font=dict(size=18, color=current_theme['font_color']),
        tickfont=dict(size=13, color=current_theme['font_color']),
        range=[0, df_grouped['Job_Count'].max() * 1.15],
        gridcolor=current_theme['grid_color'],
        linecolor=current_theme['line_color'],
        linewidth=2,
        showline=True
    ),

    # Legend styling
    legend=dict(
        title="Job Type",
        font=dict(size=13, color=current_theme['font_color']),
        bgcolor="rgba(255, 255, 255, 0.95)" if current_theme == light_theme else "rgba(30, 30, 40, 0.95)",
        bordercolor=current_theme['line_color'],
        borderwidth=1,
        x=1.02,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),

    # Hover and interactivity
    hovermode="closest",
    hoverlabel=dict(
        bgcolor="rgba(255, 255, 255, 0.9)" if current_theme == light_theme else "rgba(30, 30, 40, 0.9)",
        font_size=13,
        font_family="Helvetica, sans-serif",
        font_color=current_theme['font_color'],
        bordercolor=current_theme['line_color']
    ),

    # Bar spacing
    bargap=0.2,

    # Add interactive controls
    updatemenus=[
        # 2D/3D Toggle
        dict(
            buttons=[
                dict(
                    label="2D View",
                    method="update",
                    args=[{"type": "bar"}, {"scene": None}]
                ),
                dict(
                    label="3D View",
                    method="update",
                    args=[{"type": "bar", "z": [df_grouped['Job_Count']]}, {"scene": dict(
                        xaxis=dict(title="Industry"),
                        yaxis=dict(title="Job Type"),
                        zaxis=dict(title="Number of Jobs")
                    )}]
                )
            ],
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        # Theme Toggle
        dict(
            buttons=[
                dict(
                    label="Light Theme",
                    method="relayout",
                    args=[{
                        "plot_bgcolor": light_theme['plot_bgcolor'],
                        "paper_bgcolor": light_theme['paper_bgcolor'],
                        "font.color": light_theme['font_color'],
                        "xaxis.gridcolor": light_theme['grid_color'],
                        "yaxis.gridcolor": light_theme['grid_color'],
                        "xaxis.linecolor": light_theme['line_color'],
                        "yaxis.linecolor": light_theme['line_color'],
                        "legend.bgcolor": "rgba(255, 255, 255, 0.95)",
                        "legend.bordercolor": light_theme['line_color'],
                        "hoverlabel.bgcolor": "rgba(255, 255, 255, 0.9)",
                        "hoverlabel.bordercolor": light_theme['line_color'],
                        "hoverlabel.font.color": light_theme['font_color'],
                        "shapes[0].fillcolor": light_theme['bg_gradient']
                    }]
                ),
                dict(
                    label="Dark Theme",
                    method="relayout",
                    args=[{
                        "plot_bgcolor": dark_theme['plot_bgcolor'],
                        "paper_bgcolor": dark_theme['paper_bgcolor'],
                        "font.color": dark_theme['font_color'],
                        "xaxis.gridcolor": dark_theme['grid_color'],
                        "yaxis.gridcolor": dark_theme['grid_color'],
                        "xaxis.linecolor": dark_theme['line_color'],
                        "yaxis.linecolor": dark_theme['line_color'],
                        "legend.bgcolor": "rgba(30, 30, 40, 0.95)",
                        "legend.bordercolor": dark_theme['line_color'],
                        "hoverlabel.bgcolor": "rgba(30, 30, 40, 0.9)",
                        "hoverlabel.bordercolor": dark_theme['line_color'],
                        "hoverlabel.font.color": dark_theme['font_color'],
                        "shapes[0].fillcolor": dark_theme['bg_gradient']
                    }]
                )
            ],
            direction="down",
            showactive=True,
            x=0.3,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        # Voice Narration (Browser-dependent)
        dict(
            buttons=[
                dict(
                    label="Narrate Insights",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": 0},
                        "fromcurrent": True,
                        "transition": {"duration": 0},
                        "mode": "immediate"
                    }],
                    execute=True,
                    labeljs="() => { const msg = new SpeechSynthesisUtterance('The top industry is ' + '" +
                           df_grouped.loc[df_grouped['Job_Count'].idxmax()]['NAICS2_NAME_SHORT'] +
                           " with " + df_grouped['Job_Count'].max() + " jobs.'); window.speechSynthesis.speak(msg); }"
                )
            ],
            direction="down",
            showactive=False,
            x=0.5,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],

    # Add a slider for filtering job counts
    sliders=[
        dict(
            active=0,
            currentvalue={"prefix": "Min Job Count: "},
            pad={"t": 50},
            steps=[
                dict(
                    label=str(i),
                    method="update",
                    args=[
                        {"visible": [True if df_grouped['Job_Count'][j] >= i else False for j in range(len(df_grouped))]},
                        {"title": f"Industries with At Least {i} Jobs"}
                    ]
                ) for i in range(0, int(df_grouped['Job_Count'].max()) + 500, 500)
            ]
        )
    ]
)

# Customize the bars with animations and hover effects
fig.update_traces(
    marker=dict(
        line=dict(width=1.2, color="#2D3748")
    ),
    opacity=0.85,
    hovertemplate="<b>%{x}</b><br>Type: %{fullData.name}<br>Jobs: %{y}<extra></extra>",
    transition=dict(duration=600, easing="cubic-in-out"),
    animation=dict(
        frame=dict(duration=600, redraw=True),
        fromcurrent=True,
        transition=dict(duration=600, easing="cubic-in-out")
    )
)

# Add annotation for the highest job count
max_job = df_grouped.loc[df_grouped['Job_Count'].idxmax()]
fig.add_annotation(
    x=max_job['NAICS2_NAME_SHORT'],
    y=max_job['Job_Count'] * 1.1,
    text=f"Top: {max_job['NAICS2_NAME_SHORT']}<br>{max_job['Job_Count']} Jobs",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=-40,
    font=dict(size=13, color="#2D3748", family="Helvetica, sans-serif"),
    bgcolor="rgba(255, 255, 255, 0.85)",
    bordercolor="#2D3748",
    borderwidth=1,
    borderpad=4
)

# Add a download button for the table
fig.update_layout(
    annotations=[
        dict(
            text="Download Table as CSV",
            x=1.0,
            y=-0.2,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=12, color="#2D3748"),
            bgcolor="rgba(255, 255, 255, 0.85)",
            bordercolor="#2D3748",
            borderwidth=1,
            borderpad=4,
            clicktoshow="onoff",
            clickevent="download",
            download=dict(
                filename="job_trends_data.csv",
                content=df_grouped[['NAICS2_NAME_SHORT', 'Job_Type', 'Job_Count']].to_csv(index=False)
            )
        )
    ]
)

# Show the plot
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Debug: Check df_grouped (optional, to ensure it has the right columns)
# print("Columns in df_grouped:", df_grouped.columns)
# print("First few rows of df_grouped:\n", df_grouped.head())

# Identify the top 10 industries with the most data analyst jobs
top_industries = (
    df_grouped
    .groupby("NAICS2_NAME")["Job_Count"]
    .sum()
    .nlargest(10)
    .index
)

# Filter the dataset for top industries
df_top_industries = df_grouped[
    df_grouped["NAICS2_NAME"].isin(top_industries)
].copy()

# Reset the index to start from 0
df_top_industries = df_top_industries.reset_index(drop=True)

# Shorten industry names for better readability
industry_short_names = {
    'Professional, Scientific, and Technical Services': 'Prof. Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Admin & Waste Mgmt',
    'Health Care and Social Assistance': 'Healthcare',
    'Finance and Insurance': 'Finance',
    'Information': 'Info Tech',
    'Educational Services': 'Education',
    'Manufacturing': 'Manufacturing',
    'Retail Trade': 'Retail',
    'Accommodation and Food Services': 'Hospitality',
    'Other Services (except Public Administration)': 'Other Services',
}
df_top_industries['NAICS2_NAME_SHORT'] = (
    df_top_industries['NAICS2_NAME']
    .map(industry_short_names)
    .fillna(df_top_industries['NAICS2_NAME'])
)

# Map DATA_ANALYST_JOB to True/False for the legend
df_top_industries['Job_Type'] = df_top_industries['DATA_ANALYST_JOB'].map(
    {True: 'True', False: 'False'}
)

# Define a gradient color palette for True/False
color_palette = {
    'True': ['#4ECDC4', '#6BE6DE'],   # Teal gradient
    'False': ['#FF6B6B', '#FF8787']   # Coral Red gradient
}

# Create the bar chart
fig = px.bar(
    df_top_industries,
    x="NAICS2_NAME_SHORT",
    y="Job_Count",
    color="Job_Type",
    title="Top 10 Industries Hiring Data & Business Analysts",
    labels={
        'NAICS2_NAME_SHORT': 'Industry',
        'Job_Count': 'Number of Jobs',
        'Job_Type': 'Job Type'
    },
    barmode='group'
)

# Beautify the layout with advanced styling
fig.update_layout(
    autosize=True,
    width=1200,
    height=700,
    margin=dict(l=60, r=150, t=120, b=100),

    # solid, subtle background
    plot_bgcolor='rgba(245,245,250,1)',
    paper_bgcolor='rgba(255,255,255,1)',

    # font styling
    font=dict(family="Inter, sans-serif", size=14, color="#1A202C"),

    # title styling
    title=dict(
        text="Top 10 Industries Hiring Data & Business Analysts",
        font=dict(size=30, color="#1A202C", family="Inter, sans-serif"),
        x=0.5,
        xanchor="center",
        y=0.95,
        yanchor="top"
    ),

    # x-axis styling
    xaxis=dict(
        title="Industry",
        title_font=dict(size=18, color="#1A202C"),
        tickfont=dict(size=14, color="#4A5568"),
        tickangle=45,
        gridcolor="rgba(200,200,200,0.2)",
        linecolor="#1A202C",
        linewidth=2,
        showline=True
    ),

    # y-axis styling
    yaxis=dict(
        title="Number of Jobs",
        title_font=dict(size=18, color="#1A202C"),
        tickfont=dict(size=14, color="#4A5568"),
        range=[0, df_top_industries['Job_Count'].max() * 1.2],
        gridcolor="rgba(200,200,200,0.3)",
        linecolor="#1A202C",
        linewidth=2,
        showline=True,
        zeroline=False
    ),

    # legend styling
    legend=dict(
        title="Job Type",
        font=dict(size=13, color="#1A202C"),
        bgcolor="rgba(255,255,255,0.95)",
        bordercolor="#1A202C",
        borderwidth=1,
        x=1.02,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),

    # interactivity settings
    hovermode="closest",
    bargap=0.25,
    transition=dict(duration=800, easing="cubic-in-out")
)

# Customize the bars with gradients and hover effects
for trace in fig.data:
    trace.marker = dict(
        color=color_palette[trace.name][0],
        colorscale=[
            [0, color_palette[trace.name][0]],
            [1, color_palette[trace.name][1]]
        ],
        line=dict(width=1, color="#1A202C"),
        opacity=0.85
    )
    trace.hovertemplate = (
        "<b>%{x}</b><br>"
        "Type: %{fullData.name}<br>"
        "Jobs: %{y}<br>"
        "<extra></extra>"
    )

# Add bar labels
fig.update_traces(
    texttemplate='%{y}',
    textposition='outside',
    textfont=dict(size=12, color="#1A202C", family="Inter, sans-serif")
)

# Add annotation for the highest job count
max_job = df_top_industries['NAICS2_NAME_SHORT'].iloc[
    df_top_industries['Job_Count'].idxmax()
]
max_count = df_top_industries['Job_Count'].max()
fig.add_annotation(
    x=max_job,
    y=max_count * 1.15,
    text=f"Top: {max_job}<br>{max_count} Jobs",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=-40,
    font=dict(size=13, color="#1A202C", family="Inter, sans-serif"),
    bgcolor="rgba(255,255,255,0.95)",
    bordercolor="#1A202C",
    borderwidth=1,
    borderpad=4
)

# Add shadow effect to the title
fig.add_annotation(
    x=0.5,
    y=0.95,
    xref="paper",
    yref="paper",
    text="Top 10 Industries Hiring Data & Business Analysts",
    showarrow=False,
    font=dict(size=30, color="rgba(0,0,0,0.05)", family="Inter, sans-serif"),
    xanchor="center",
    yanchor="top",
    yshift=-2,
    xshift=2
)

# Show the plot
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Define target industries
target_industries = ["Finance and Insurance", "Health Care and Social Assistance"]

# Filter the original DataFrame (df) instead of df_grouped
df_remote = eda[eda["NAICS2_NAME"].isin(target_industries)]

# Count occurrences of each remote type in these industries
df_remote_grouped = df_remote.groupby("REMOTE_TYPE_NAME").size().reset_index(name="Count")

# Define a modern color palette
color_map = {
    "Remote": "#6AB1E9",  # Soft Sky Blue
    "On-Site": "#EF767A",  # Soft Coral
    "Hybrid": "#49C6B7"   # Teal-Green
}

# Create the pie chart
fig = px.pie(df_remote_grouped, 
             names="REMOTE_TYPE_NAME", 
             values="Count", 
             title="Remote vs. On-Site Data Analyst Jobs in Finance & Healthcare",
             color="REMOTE_TYPE_NAME", 
             color_discrete_map=color_map)

# Beautify the layout
fig.update_layout(
    # Sizing and margins
    autosize=True,
    width=900,  # Compact width for pie charts
    height=700,  # Taller height to accommodate title and legend
    margin=dict(l=50, r=50, t=120, b=50),  # Adjusted margins for title

    # Background styling
    plot_bgcolor='rgba(240, 240, 245, 1)',  # Light gray background
    paper_bgcolor='rgba(255, 255, 255, 1)',  # White paper background
    font=dict(family="Poppins, sans-serif", size=14, color="#2D3436"),  # Modern font

    # Title styling
    title=dict(
        text="Remote vs. On-Site Data Analyst Jobs in Finance & Healthcare",
        font=dict(size=24, color="#2D3436", family="Poppins, sans-serif"),
        x=0.5,
        xanchor="center",
        y=0.95,
        yanchor="top"
    ),

    # Legend styling
    legend=dict(
        title="Remote Type",
        font=dict(size=13, color="#2D3436"),
        bgcolor="rgba(255, 255, 255, 0.95)",
        bordercolor="#2D3436",
        borderwidth=1,
        x=1.05,  # Position outside
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),

    # Hover and interactivity
    hovermode="closest",
    hoverlabel=dict(
        bgcolor="rgba(255, 255, 255, 0.9)",
        font_size=13,
        font_family="Poppins, sans-serif",
        font_color="#2D3436",
        bordercolor="#2D3436"
    ),

    # Add a subtle gradient background effect
    shapes=[
        dict(
            type="rect",
            x0=0, y0=0, x1=1, y1=1,
            xref="paper", yref="paper",
            fillcolor="rgba(220, 221, 255, 0.2)",  # Light gradient overlay
            line=dict(width=0),
            layer="below"
        )
    ]
)

# Customize the pie chart slices
fig.update_traces(
    # Add percentages and labels
    textinfo="percent+label",
    textfont=dict(size=14, color="#2D3436", family="Poppins, sans-serif"),
    textposition="inside",  # Place labels inside for a cleaner look

    # Add a subtle border to slices
    marker=dict(
        line=dict(color="#2D3436", width=1.5)
    ),

    # Pull out the largest slice slightly for emphasis
    pull=[0.1 if count == df_remote_grouped['Count'].max() else 0 for count in df_remote_grouped['Count']],

    # Add a slight opacity for a softer look
    opacity=0.9,

    # Custom hover template
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>",

    # Add a rotation for better presentation
    rotation=45,

    # Add a subtle shadow effect to slices (simulated with opacity and border)
    sort=False  # Keep the order as in the data
)

# Add an annotation for the largest slice
max_slice = df_remote_grouped.loc[df_remote_grouped['Count'].idxmax()]
fig.add_annotation(
    x=0.5,
    y=0.1,
    xref="paper",
    yref="paper",
    text=f"Dominant Type: {max_slice['REMOTE_TYPE_NAME']}<br>{max_slice['Count']} Jobs ({fig.data[0]['values'].tolist().index(max_slice['Count'])*100/sum(fig.data[0]['values']):.1f}%)",
    showarrow=False,
    font=dict(size=13, color="#2D3436", family="Poppins, sans-serif"),
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="#2D3436",
    borderwidth=1,
    borderpad=4
)

# Add a shadow effect to the title (simulated with a duplicate text)
fig.add_annotation(
    x=0.5,
    y=0.95,
    xref="paper",
    yref="paper",
    text="Remote vs. On-Site Data Analyst Jobs in Finance & Healthcare",
    showarrow=False,
    font=dict(size=24, color="rgba(0, 0, 0, 0.05)", family="Poppins, sans-serif"),
    xanchor="center",
    yanchor="top",
    yshift=-2,
    xshift=2
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1) Trim out the 1–99th percentile so outliers don’t skew the means
low, high = eda['SALARY'].quantile([0.01, 0.99])
eda_trim = eda[(eda['SALARY'] >= low) & (eda['SALARY'] <= high)]

# 2) Compute mean salary by remote type
means = eda_trim.groupby('REMOTE_TYPE_NAME')['SALARY'].mean().loc[['Remote', 'Not Remote', 'Hybrid Remote']]

# 3) Plot with value labels
plt.figure(figsize=(10, 7))
bars = means.plot.bar(color=['#4C72B0', '#55A868', '#C44E52'])
plt.title('Average Salary by Remote Type (1–99th pct)')
plt.ylabel('Mean Annual Salary')
plt.xticks(rotation=0)

# Add value labels on top of each bar
for bar in bars.patches:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{int(bar.get_height())}', 
             ha='center', va='bottom')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Trim outliers
low, high = eda['SALARY'].quantile([0.01, 0.99])
eda_trim = eda[(eda['SALARY'] >= low) & (eda['SALARY'] <= high)]

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(eda_trim['SALARY'], bins=30, color='#4C72B0', edgecolor='black')
plt.title('Salary Distribution (1–99th pct)')
plt.xlabel('Annual Salary')
plt.ylabel('Frequency')

In [ ]:
import plotly.express as px
import pandas as pd

# Define target industries (same as before)
target_industries = ["Finance and Insurance", "Health Care and Social Assistance"]

# Filter the original DataFrame for these industries
# Assuming 'df' has a 'SALARY' column (from our regression analysis context)
df_target = eda[eda["NAICS2_NAME"].isin(target_industries)].copy()

# Map DATA_ANALYST_JOB to meaningful labels
df_target['Job_Category'] = df_target['DATA_ANALYST_JOB'].map({True: 'Data Analyst', False: 'Business Analyst'})

# Group by Job_Category and REMOTE_TYPE_NAME, calculate average salary
df_salary_grouped = df_target.groupby(['Job_Category', 'REMOTE_TYPE_NAME'])['SALARY'].mean().reset_index(name='Avg_Salary')

# Create a grouped bar chart
fig = px.bar(df_salary_grouped,
             x='Job_Category',
             y='Avg_Salary',
             color='REMOTE_TYPE_NAME',
             title="Average Salary by Remote Type and Job Category (Finance & Healthcare)",
             labels={'Job_Category': 'Job Category', 'Avg_Salary': 'Average Salary ($)', 'REMOTE_TYPE_NAME': 'Remote Type'},
             barmode='group',
             color_discrete_map={"Remote": "#6AB1E9", "On-Site": "#EF767A", "Hybrid": "#49C6B7"})

# Beautify the layout
fig.update_layout(
    width=900,
    height=600,
    plot_bgcolor='rgba(240, 240, 245, 1)',
    paper_bgcolor='rgba(255, 255, 255, 1)',
    font=dict(family="Poppins, sans-serif", size=14, color="#2D3436"),
    title=dict(
        text="Average Salary by Remote Type and Job Category (Finance & Healthcare)",
        font=dict(size=24, color="#2D3436"),
        x=0.5,
        xanchor="center"
    ),
    xaxis=dict(
        title="Job Category",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title="Average Salary ($)",
        title_font=dict(size=18),
        tickfont=dict(size=14),
        gridcolor="rgba(200, 200, 200, 0.3)"
    ),
    legend=dict(
        title="Remote Type",
        font=dict(size=13),
        x=1.05,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),
    bargap=0.2
)

# Add value labels on top of bars
fig.update_traces(
    text=df_salary_grouped['Avg_Salary'].round(2),
    textposition='outside',
    textfont=dict(size=12, color="#2D3436")
)
